In [2]:
from azureml.core import Workspace 

ws = Workspace(subscription_id="fc1c5e68-95c2-4bce-9ff9-5bd8442fb921", resource_group="titanic-api", workspace_name="titanic-ws")
default_ds = ws.get_default_datastore()

print(ws.name, "workspace loaded.")
print(default_ds.name, "datastore loaded.")

titanic-ws workspace loaded.
workspaceblobstore datastore loaded.


# Modelling

In [8]:
import pandas as pd
import titanic_functions as tfunc
from sklearn.preprocessing import StandardScaler

df2 = pd.read_csv("../dataset/titanic_data.csv")
df2 = tfunc.clean_df(df2)

X = df2.drop("survived", axis=1)
y = df2["survived"]

scaler = StandardScaler()
scaler.fit(X)

Xsc = scaler.transform(X)

In [6]:
# Split the dataset in two equal parts
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(Xsc, y, test_size=0.2, random_state=0)

In [7]:
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

gnb = GaussianNB()
rfc = RandomForestClassifier()
lr = LogisticRegression()

In [8]:
gnb = gnb.fit(X_train, y_train)
rfc = rfc.fit(X_train, y_train)
lr = lr.fit(X_train, y_train)

In [9]:
pred_gnb = gnb.predict(X_test)
pred_rfc = rfc.predict(X_test)
pred_lr = lr.predict(X_test)

In [10]:
from sklearn.metrics import confusion_matrix, plot_confusion_matrix, accuracy_score, precision_score, recall_score, roc_curve, roc_auc_score, classification_report, f1_score
import numpy as np

models = {"Gaussian NB": pred_gnb, "Random Forest Classifier": pred_rfc, "Logistic Regression": pred_lr}
metrics = {}

for name, pred in models.items():
    print(name+":")

    print("\tAccuracy:", np.round(accuracy_score(y_test, pred),4))
    print("\tPrecision:", np.round(precision_score(y_test, pred),4))
    print("\tRecall:", np.round(recall_score(y_test, pred),4))
    print("\tF1-Score:", np.round(f1_score(y_test, pred),4))
    print()

    m = {"Accuracy:": np.round(accuracy_score(y_test, pred),4),
        "Precision:": np.round(precision_score(y_test, pred),4),
        "Recall:": np.round(recall_score(y_test, pred),4),
        "F1-Score:": np.round(f1_score(y_test, pred),4)}
    metrics[name] = m 

Gaussian NB:
	Accuracy: 0.7023
	Precision: 0.5897
	Recall: 0.8679
	F1-Score: 0.7023

Random Forest Classifier:
	Accuracy: 0.7863
	Precision: 0.7451
	Recall: 0.717
	F1-Score: 0.7308

Logistic Regression:
	Accuracy: 0.8282
	Precision: 0.7699
	Recall: 0.8208
	F1-Score: 0.7945



In [11]:
import joblib

best_model = lr
best_model_name = "Logistic Regression"
best_model_metrics = metrics[best_model_name]

os.makedirs("model_folder", exist_ok=True)
joblib.dump(best_model, "model_folder/titanic-api-model.pkl")
joblib.dump(scaler, "model_folder/titanic-api-scaler.pkl")

default_ds.upload_files(["model_folder/titanic-api-model.pkl", "model_folder/titanic-api-scaler.pkl"], target_path="model_folder", overwrite=True)

Uploading an estimated of 2 files
Uploading model_folder/titanic-api-model.pkl
Uploaded model_folder/titanic-api-model.pkl, 1 files out of an estimated total of 2
Uploading model_folder/titanic-api-scaler.pkl
Uploaded model_folder/titanic-api-scaler.pkl, 2 files out of an estimated total of 2
Uploaded 2 files


$AZUREML_DATAREFERENCE_ea0fee9a0a034a629ca616c5852d0d86

In [12]:
from azureml.core import Model 

In [13]:
Model.register(ws, "model_folder/titanic-api-model.pkl", "titanic-api-model", tags=best_model_metrics)

Registering model titanic-api-model


Model(workspace=Workspace.create(name='titanic-ws', subscription_id='fc1c5e68-95c2-4bce-9ff9-5bd8442fb921', resource_group='titanic-api'), name=titanic-api-model, id=titanic-api-model:10, version=10, tags={'Accuracy:': '0.8282', 'Precision:': '0.7699', 'Recall:': '0.8208', 'F1-Score:': '0.7945'}, properties={})

In [14]:
Model.register(ws, "model_folder/scaler.pkl", "titanic-api-scaler", tags={"Scaler":"Titanic Input Scaler"})

Registering model titanic-api-scaler


Model(workspace=Workspace.create(name='titanic-ws', subscription_id='fc1c5e68-95c2-4bce-9ff9-5bd8442fb921', resource_group='titanic-api'), name=titanic-api-scaler, id=titanic-api-scaler:7, version=7, tags={'Scaler': 'Titanic Input Scaler'}, properties={})

In [37]:
# model = Model.get_model_path("titanic-api-model")
# scaler = Model.get_model_path("titanic-api-scaler")

# DEPLOYMENT

In [16]:
%%writefile model_folder/score.py

import json
import numpy as np
import os
import joblib


########### CUSTOM FUNCTIONS ###################

import pandas as pd

# Replacing all titles with mr, mrs, miss, master
def replace_titles(x):
    title=x['title']
    if title in ['Don', 'Major', 'Capt', 'Jonkheer', 'Rev', 'Col']:
        return 'Mr'
    elif title in ['Countess', 'Mme']:
        return 'Mrs'
    elif title in ['Mlle', 'Ms']:
        return 'Miss'
    elif title =='Dr':
        if x['sex']=='Male':
            return 'Mr'
        else:
            return 'Mrs'
    else:
        return title

def clean_df(df, verbose=False):

    if not isinstance(df, pd.DataFrame):
        df = pd.DataFrame(df, columns = ["pclass", "name", "sex", "age", "sibsp", "parch", "fare", "cabin", "embarked"]).astype({"pclass":"int64","age":"float64","sibsp":"int64","parch":"int64","fare":"float64"})
    df.columns = df.columns.str.lower()

    # Drop "boat", "body", "home.dest" and "ticket". The first two hold information if the passenger survived (boat) or if it didn't and the body was recovered (body).
    try:
        df = df.drop(["boat", "body", "home.dest", "ticket"], axis=1)
    except KeyError:
        if verbose:
            print("Any of these features are not in the dataframe: boat, body, home.dest, ticket")


    # Just a few observations, drop them
    df.drop(df[(pd.isnull(df["embarked"]))].index, inplace=True)

    # Replace NULL with median
    df["fare"].fillna(df["fare"].median(), inplace=True)
    df["age"].fillna(df["age"].median(), inplace=True)

    # from: https://triangleinequality.wordpress.com/2013/09/08/basic-feature-engineering-with-the-titanic-data/

    # Extract titles
    df["title"] = df["name"].str.extract(r'(Mrs|Mr|Master|Miss|Major|Rev|Dr|Ms|Mlle|Col|Capt|Mme|Countess|Don|Jonkheer)')
    df['title']=df.apply(replace_titles, axis=1)
    df = df.drop("name", axis=1)

    # Extract Deck
    df['deck'] = df["cabin"].str[0].fillna("Unknown")
    df = df.drop("cabin", axis=1)

    # Create new family_size column
    df['family_size'] = df['sibsp']+df['parch'] + 1 #counting the passenger itself
    df['fare_per_person'] = df['fare']/df['family_size']
    df['alone'] = df['family_size'].apply(lambda x: 1 if x==1 else 0)

    # Because why not
    df['age*class'] = df['age']*df['pclass']

####################################### CATEGORIES #######################################
# Used to create dummies when passing in one single observation

    # dummy categories: "sex", "embarked", "title", "deck"
    sex = ["male", "female"]
    embarked = ["S", "C", "Q"]
    title = ["Mr", "Mrs", "Miss", "Master"]
    deck = ["A", "B", "C", "D", "E", "F", "G", "T", "Unknown"]    

    df["sex"] = df["sex"].astype(pd.CategoricalDtype(sex))
    df["embarked"] = df["embarked"].astype(pd.CategoricalDtype(embarked))
    df["title"] = df["title"].astype(pd.CategoricalDtype(title))
    df["deck"] = df["deck"].astype(pd.CategoricalDtype(deck))
###############################################################################################


    # Create final dataframe with dummies
    df2 = pd.get_dummies(df, columns=["sex", "embarked", "title", "deck"], prefix="dummy")

    return df2
    

################### DEPLOYMENT ##################



def init():
    global model
    global scaler
    model_path = './model_folder/titanic-api-model.pkl'
    model = joblib.load(model_path)

    scaler_path = './model_folder/titanic-api-scaler.pkl'
    scaler = joblib.load(scaler_path)

def run(data):

    #ws = Workspace(subscription_id="fc1c5e68-95c2-4bce-9ff9-5bd8442fb921", resource_group="titanic-api", workspace_name="titanic-ws")

    try:
        data = json.loads(data)['data']
        data=list(clean_df(data).values[0])
        data = scaler.transform([data])
        result = model.predict(data)
        # You can return any data type, as long as it is JSON serializable.
        return result.tolist()
    except Exception as e:
        error = str(e)
        return error


Overwriting model_folder/score.py


In [3]:
# Deployment Env

from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies

# Create an environment and add conda dependencies to it
myenv = Environment(name="myenv")
# Enable Docker based environment
myenv.docker.enabled = True
# Build conda dependencies
myenv.python.conda_dependencies = CondaDependencies.create(conda_packages=['scikit-learn'],
                                                           pip_packages=['azureml-defaults'])
                                                        
print("Deployment")

Deployment


In [5]:
# Deployment config
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig
# from azureml.core import Model 

# model = ws.models["titanic-api-model"]

inference_config = InferenceConfig(source_directory="model_folder",
                                   entry_script="score.py",
                                   environment=myenv)

# deploy_config = AciWebservice.deploy_configuration(cpu_cores=0.1, memory_gb=0.5)

# service_name = "titanic-api"

# service = Model.deploy(ws, service_name, [model], inference_config, deploy_config)
# service.wait_for_deployment(True)
# print(service.state)

service = AciWebservice(ws, "titanic-api")
service.update(inference_config=inference_config)
print(service.state)


AttributeError: 'NoneType' object has no attribute 'lower'

In [6]:
# TEST ENDPOINT
endpoint = service.scoring_uri
print(endpoint)

http://a278d689-be85-4c99-a512-1b715ae287db.uksouth.azurecontainer.io/score


In [9]:
i=1

x_new = list(df2.drop("survived",axis=1).iloc[i].values)

In [10]:
import json
import requests

# Convert the array to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})

# Set the content type
headers = { 'Content-Type':'application/json' }


In [15]:
requests.post(endpoint, input_json, headers = headers)

ConnectionError: HTTPConnectionPool(host='a278d689-be85-4c99-a512-1b715ae287db.uksouth.azurecontainer.io', port=80): Max retries exceeded with url: /score (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000159616DB520>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond'))

In [12]:
print("Predicted:", requests.post(endpoint, input_json, headers = headers).json()[0])
print("Actual:", df2["survived"].iloc[i])

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [23]:
import titanic_functions as tfunc
import json

data=[[1,"Ms","Female", 0,0,0,30000,"C", "S"]]

data=list(tfunc.clean_df(data).values[0])

input_json = json.dumps({"data": data})
headers = { 'Content-Type':'application/json' }

requests.post(endpoint, input_json, headers = headers).json()[0]


1

In [21]:
requests.post(endpoint, input_json, headers = headers).json()

[0]